In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel("DEBUG")

In [4]:
import torchChar
import torch.nn as nn
import torch.nn.functional as F

../src\torchChar\prepare_data.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
from argparse import ArgumentParser

In [14]:
parser = ArgumentParser()
parser = torchChar.AliceModel.add_model_args(parser)
args = parser.parse_known_args()[0]
args.lr = 1e-4
args.n_radicals = len(torchChar.Radicals())
args.n_tones = len(torchChar.Tones())
args.n_consonants = len(torchChar.Consonants())
args.n_vowels = len(torchChar.Vowels())
model = torchChar.AliceModel(args)

In [16]:
model.fc1.in_features

204800

In [9]:
from argparse import Namespace

In [10]:
ns = Namespace()

In [11]:
ns.aa=2

In [12]:
ns.aa

2

In [7]:
a.update([1,2,3, 4])

In [8]:
a

{1, 2, 3, 4}